# Exercise 3: Linear Regression on a Synthetic Dataset

1. Generate synthetic data with an intercept and 5 features (100 samples).
2. Implement a gradient descent solver.
3. Compute solutions by closed‐form OLS, scikit‐learn, and gradient descent.
4. Compare the parameter estimates to verify they coincide.
5. Feature–Response correlation and $R^2$ on Training Data.

# 1) Generate synthetic data

In [ ]:
import numpy as np
np.random.seed(0)

n_samples = 100
n_features = 5

"""
True parameters (intercept + 5 coefficients)
"""
beta_true = np.array([2.0, -1.5, 3.0, 0.0, 1.0, -2.0])

# Design matrix: first column of ones, then random features
X_feat = np.random.randn(n_samples, n_features)
X = np.hstack([np.ones((n_samples,1)), X_feat])  # shape (100,6)

# Generate targets with Gaussian noise
noise = np.random.randn(n_samples) * 0.5
y = X.dot(beta_true) + noise

print("X shape:", X.shape)
print("\nFirst 5 rows of X:\n", X[:5])
print("\nFirst 5 targets y:\n", y[:5])
print("\nTrue coefficients:\n", beta_true)

## 2) Gradient Descent Implementation

In [ ]:
def gradient_descent(X, y, lr=0.01, n_iters=1000, verbose=False):
    """
    Solve for theta in linear regression by batch gradient descent.
    X: design matrix (n x d), y: targets (n, )
    lr: learning rate, n_iters: number of iterations
    Returns theta (d,)
    """
    n, d = X.shape
    theta = np.zeros(d)
    for it in range(n_iters):
        grad = (X.T.dot(X.dot(theta) - y)) / n
        theta -= lr * grad
        if verbose and it % (n_iters//5) == 0:
            loss = np.mean((X.dot(theta) - y)**2)
            print(f"Iter {it:4d}, MSE={loss:.4f}")
    return theta

## 3) Compute Solutions: OLS, scikit‐learn, and Gradient Descent

In [ ]:
# 3a) Closed‐form OLS solution
beta_closed = np.linalg.pinv(X.T.dot(X)).dot(X.T).dot(y)

# 3b) scikit‐learn solution
from sklearn.linear_model import LinearRegression
lr_model = LinearRegression(fit_intercept=False)
lr_model.fit(X, y)
beta_sklearn = lr_model.coef_

# 3c) Gradient Descent solution
beta_gd = gradient_descent(X, y, lr=0.1, n_iters=5000)

print("True theta:        ", beta_true)
print("Closed‐form theta: ", np.round(beta_closed, 4))
print("sklearn theta:     ", np.round(beta_sklearn, 4))
print("GD solution theta: ", np.round(beta_gd, 4))

## 4) Comparison and Discussion
- The parameter estimates from closed‐form OLS, scikit‐learn, and gradient descent should all be very close (or identical) to each other.
- They shoudl be close to the true `beta_true`.
- Minor differences arise from noise and finite iterations in GD.
- If GD hasn't converged, adjust the learning rate `lr` or increase `n_iters`.

## 5) Feature–Response Correlation and $R^2$ on Training Data
Compute the Pearson correlation between each feature (excluding the intercept) and the predicted response, and evaluate the coefficient of determination $R^2$.

In [ ]:
from sklearn.metrics import r2_score

# 5a) Predicted response via gradient descent solution
y_pred = X.dot(beta_gd)

# 5b) Pearson correlation for each feature (skip intercept at col 0)
print("Feature and y_pred correlations:")
for j in range(1, X.shape[1]):
    corr = np.corrcoef(X[:, j], y_pred)[0, 1]
    print(f"\tFeature {j}: corr = {corr:.4f}")

# 5c) R^2 score on training data
r2 = r2_score(y, y_pred)
print(f"\nR^2 on training set: {r2:.4f}")